# Vakyansh + Hugging Face : Hindi Speech To text Demo

## Install requirements

In [ ]:
%%capture

!apt-get -y install sox ffmpeg
!pip install transformers --upgrade 
!pip install ffmpeg-python sox

!wget https://raw.githubusercontent.com/harveenchadha/bol/main/demos/colab/record.py

!pip install pyctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

## Load Hindi Model

In [ ]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, AutoModelForCTC

def load_model():
    # load pretrained model
    processor_with_lm = Wav2Vec2ProcessorWithLM.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    model = AutoModelForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    return processor_with_lm, processor, model

processor_with_lm, processor, model = load_model()


In [ ]:
def parse_transcription(wav_file, use_lm=False):
    # load audio    
    audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=16_000, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
     #.cpu()
    #predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    #transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)

    #transcription = processor.decode(logits)
    if use_lm: 
        with torch.no_grad():
            logits = model(input_values).logits.cpu().detach().numpy()[0]            
        transcription = processor_with_lm.decode(logits).text
        return transcription.replace('<s>','')
    else:
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
        
        return transcription

## Record file using colab

In [ ]:
from record import record_audio

record_audio('test')

## Run Model on recorded file

In [ ]:
output_with_lm = parse_transcription('test.wav', True)
output = parse_transcription('test.wav')

In [ ]:
output, output_with_lm

('और सुनाये क्या हलचाल है', 'और्सुना  ये क्या हलचल है ')